In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
dir = "C:/Users/foresight_User/Code/測試資料/"


In [4]:
from  pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
import torch

In [5]:

CH_TO = pd.read_csv(dir+"CH_TO.csv").dropna()
feature = CH_TO.drop(columns=["Y","Context Name"])#.to_numpy()
target = CH_TO["Y"]#.to_numpy()
#加入CROSS項目，有時候會顯著，有時候不會顯著，明顯受到切資料的影響， 但CROSS BEST的模型確實會比較好


# feature = pd.read_csv(dir+"wb/Indicator_Data(1).csv").drop(columns=["Unnamed: 74","Context Name"])#.to_numpy()
# target = pd.read_csv(dir+"wb/Metrology_Data(1).csv")["Point1"]#.to_numpy()

# data = pd.read_excel(dir+"Array7_N808測試資料集/Model_TJN808XK_SAMP75.xlsx").drop(columns=["CONTEXTID"]).dropna(axis=0)
# feature = data.drop(columns=list(filter(lambda x: "T2_CD" in x,data.columns)))#.to_numpy()
# target = data["T2_CD01_Mean"]#.to_numpy()

columns_name = list(feature.columns) # or np.arange(0,feature.shape[1])
#
# 分割數據集
X_train, X_test, y_train, y_test = train_test_split(feature.to_numpy(), target.to_numpy().reshape(-1,1), test_size=0.3)


# Load data

#X_valid, _, y_valid, _ = train_test_split(X_train, y_train, test_size=0.2)
#X_valid, _, y_valid, _ = train_test_split(X_train, y_train.reshape(-1,1), test_size=0.3)

TabNetWrapper()


TabNetWrapper()

In [35]:
from torchensemble.gradient_boosting import GradientBoostingRegressor
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetRegressor

# from torchensemble import VotingRegressor

# class TabNetWrapper(nn.Module):
#     def __init__(self, tabnet_model):
#         super(TabNetWrapper, self).__init__()
        
#         # 複製 TabNetRegressor 的權重
#         self.tabnet = tabnet_model
        
#     def forward(self, x):
#         # 在 forward 函數中定義模型的前向傳播邏輯
#         return self.tabnet(x)

# # 假設你已經創建了 TabNetRegressor 的實例 tabnet_model
# # 然後將它傳遞給 TabNetWrapper
# tabnet_model = TabNetRegressor(input_dim=X_train.shape[1], output_dim=1)
# model = TabNetWrapper(tabnet_model)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(in_features=30, out_features=16)
        self.hidden_1 = nn.Linear(in_features=16, out_features=16)
        self.output = nn.Linear(in_features=16, out_features=1)
        
    def forward(self, x):
        x = torch.nn.functional.relu(self.input(x))
        x = torch.nn.functional.relu(self.hidden_1(x))
        return self.output(x)
    
    
model = Net()

# 輸出模型結構
print(model)
# 假設 feature 和 target 是 pandas DataFrame 或 numpy array

# 將 feature 和 target 轉換成 TensorDataset
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                              torch.tensor(y_train, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                             torch.tensor(y_test, dtype=torch.float32))

# 定義 DataLoader
batch_size = 32  # 可以根據需要調整批次大小
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



# 訓練TabNet模型
batch_size = int(X_train.shape[0]/3)
#virtual_batch_size = int(batch_size/3)
#model = TabNetRegressor()
# model.fit(
#     X_train, y_train,
#     eval_set=[(X_train, y_train)],
#     eval_name=['train'],
#     eval_metric=['rmsle', 'mae', 'rmse', 'mse'],
#     max_epochs=2000,
#     patience=100,
#     batch_size=batch_size,
#     #virtual_batch_size=virtual_batch_size
# )
ensemble = GradientBoostingRegressor(model,n_estimators=100,cuda=False)
learning_rate = 0.001
epochs = 1000

ensemble.set_optimizer(
    "Adam",                                 # type of parameter optimizer
    lr=learning_rate,                       # learning rate of parameter optimizer
    weight_decay=5e-4,              # weight decay of parameter optimizer
)

# Set the learning rate scheduler
ensemble.set_scheduler(
    "CosineAnnealingLR",                    # type of learning rate scheduler
    T_max=epochs,                           # additional arguments on the scheduler
)

# Train the ensemble
ensemble.fit(
    train_loader,
    epochs=epochs,                          # number of training epochs
)


Net(
  (input): Linear(in_features=30, out_features=16, bias=True)
  (hidden_1): Linear(in_features=16, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
)


In [48]:
from sklearn.metrics import r2_score
r2_score(y_test, np.array(ensemble.predict(X_test)))

0.1036180268267376

In [42]:
from sklearn.metrics import r2_score
r2_score(y_test, model.predict(X_test))

(191, 30)

In [4]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

# # TabNetPretrainer
# unsupervised_model = TabNetPretrainer(
#     optimizer_fn=torch.optim.Adam,
#     optimizer_params=dict(lr=2e-2),
#     mask_type='entmax' # "sparsemax"
# )

# unsupervised_model.fit(
#     X_train=X_train,
#     pretraining_ratio=0.8,
# )

reg = TabNetRegressor(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, # how to use learning rate scheduler
                      "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='sparsemax' # This will be overwritten if using pretrain model
)

reg.fit(
    X_train=X_train, y_train=y_train,
    eval_metric=['rmse'],
    #from_unsupervised=unsupervised_model
)

c:\Users\foresight_User\anaconda3\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.0     |  0:00:00s
epoch 1  | loss: 0.0     |  0:00:00s
epoch 2  | loss: 0.0     |  0:00:00s
epoch 3  | loss: 0.0     |  0:00:00s
epoch 4  | loss: 0.0     |  0:00:00s
epoch 5  | loss: 0.0     |  0:00:00s
epoch 6  | loss: 0.0     |  0:00:00s
epoch 7  | loss: 0.0     |  0:00:00s
epoch 8  | loss: 0.0     |  0:00:00s
epoch 9  | loss: 0.0     |  0:00:00s
epoch 10 | loss: 0.0     |  0:00:00s
epoch 11 | loss: 0.0     |  0:00:00s
epoch 12 | loss: 0.0     |  0:00:00s
epoch 13 | loss: 0.0     |  0:00:00s
epoch 14 | loss: 0.0     |  0:00:00s
epoch 15 | loss: 0.0     |  0:00:00s
epoch 16 | loss: 0.0     |  0:00:00s
epoch 17 | loss: 0.0     |  0:00:00s
epoch 18 | loss: 0.0     |  0:00:00s
epoch 19 | loss: 0.0     |  0:00:00s
epoch 20 | loss: 0.0     |  0:00:00s
epoch 21 | loss: 0.0     |  0:00:00s
epoch 22 | loss: 0.0     |  0:00:00s
epoch 23 | loss: 0.0     |  0:00:00s
epoch 24 | loss: 0.0     |  0:00:00s
epoch 25 | loss: 0.0     |  0:00:00s
epoch 26 | loss: 0.0     |  0:00:00s
e

c:\Users\foresight_User\anaconda3\lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


-7638.519027486152

array([[1675.7844  ],
       [1095.1241  ],
       [1675.6028  ],
       [ -21.589806],
       [1683.2109  ],
       [1683.123   ],
       [1093.7427  ],
       [1675.7667  ],
       [1675.899   ],
       [1675.7941  ],
       [1095.1273  ],
       [1683.0305  ],
       [1133.1093  ],
       [1683.1528  ],
       [1683.1223  ],
       [1675.8704  ],
       [1132.1826  ],
       [1095.1241  ],
       [1095.795   ],
       [1683.0759  ],
       [1683.5408  ],
       [1675.6655  ],
       [1094.1923  ],
       [1675.668   ],
       [1675.8245  ],
       [1254.2139  ],
       [1095.795   ],
       [1095.795   ],
       [1683.5935  ],
       [1682.9442  ],
       [1683.6235  ],
       [1683.0275  ],
       [1704.561   ],
       [1095.1241  ],
       [1683.5459  ],
       [1095.1241  ],
       [1682.9272  ],
       [1675.8809  ],
       [1675.9208  ],
       [1683.5334  ],
       [1096.2456  ],
       [1254.2892  ],
       [1675.7129  ],
       [1675.7463  ],
       [1095.1241  ],
       [10

In [12]:
unsupervised_model.fit(X_train.to_numpy())

epoch 0  | loss: 0.0     |  0:00:00s
epoch 1  | loss: 0.0     |  0:00:00s
epoch 2  | loss: 0.0     |  0:00:00s
epoch 3  | loss: 0.0     |  0:00:00s
epoch 4  | loss: 0.0     |  0:00:00s
epoch 5  | loss: 0.0     |  0:00:00s
epoch 6  | loss: 0.0     |  0:00:00s
epoch 7  | loss: 0.0     |  0:00:00s
epoch 8  | loss: 0.0     |  0:00:00s
epoch 9  | loss: 0.0     |  0:00:00s
epoch 10 | loss: 0.0     |  0:00:00s
epoch 11 | loss: 0.0     |  0:00:00s
epoch 12 | loss: 0.0     |  0:00:00s
epoch 13 | loss: 0.0     |  0:00:00s
epoch 14 | loss: 0.0     |  0:00:00s
epoch 15 | loss: 0.0     |  0:00:00s
epoch 16 | loss: 0.0     |  0:00:00s
epoch 17 | loss: 0.0     |  0:00:00s
epoch 18 | loss: 0.0     |  0:00:00s
epoch 19 | loss: 0.0     |  0:00:00s
epoch 20 | loss: 0.0     |  0:00:00s
epoch 21 | loss: 0.0     |  0:00:00s
epoch 22 | loss: 0.0     |  0:00:00s
epoch 23 | loss: 0.0     |  0:00:00s
epoch 24 | loss: 0.0     |  0:00:00s
epoch 25 | loss: 0.0     |  0:00:00s
epoch 26 | loss: 0.0     |  0:00:00s
e

c:\Users\foresight_User\anaconda3\lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


In [15]:
unsupervised_model.prepare_target(y_test)

192    14048.4
227    14086.0
555    14089.8
340    13975.4
560    13949.0
        ...   
139    13905.6
68     13896.2
618    14202.6
536    14201.0
513    14303.4
Name: Y, Length: 128, dtype: float64

In [9]:
y_test

192    14048.4
227    14086.0
555    14089.8
340    13975.4
560    13949.0
        ...   
139    13905.6
68     13896.2
618    14202.6
536    14201.0
513    14303.4
Name: Y, Length: 128, dtype: float64